# Importing all the packages

In [2]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 120)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif


## Loading and reading the csv file 

In [6]:
path = "C:/Users/Anshika/Downloads/Data_Engineer_test_and_data__1_"
employee = path+"/Data manipulation test - Employees.csv"
employee_path = pd.read_csv(employee)
employee_path.head()

,id,gender,region,salary,DOB,industries
0,101,Female,Isle of Man,13000,23937,Transportation and storage
1,98,Female,Northern Ireland,127000,28619,Other service activities
2,99,Male,Northern Ireland,123000,22342,Real estate activities
3,96,Female,Wales,49000,24407,"Water supply, sewerage, waste management and r..."
4,104,Male,East Midlands,70000,21511,Human health and social work activities


In [9]:
fundPrice = path+"/Data manipulation test - Fund prices.csv"
fundPrice_path = pd.read_csv(fundPrice)
fundPrice_path.head()

,id,amount_on,fund_id,unit_value_pounds
0,1,24/02/2020,72,1.44
1,2,24/02/2020,73,1.23
2,3,24/02/2020,74,1.43
3,4,24/02/2020,75,1.23
4,5,24/02/2020,76,1.24


In [11]:
fund = path+"/Data manipulation test - Funds.csv"
fund_path = pd.read_csv(fund)
fund_path.head()

,id,Fund Name
0,72,Low risk fund
1,73,High risk fund
2,74,Moderate risk fund
3,75,Government bond fund
4,76,Investment fund


#### Merging fund and fund_price files

In [15]:
fundFundPrice= pd.merge(fundPrice_path, fund_path, left_on='fund_id', right_on='id', how='left')
fundFundPrice.head()

,id_x,amount_on,fund_id,unit_value_pounds,id_y,Fund Name
0,1,24/02/2020,72,1.44,72,Low risk fund
1,2,24/02/2020,73,1.23,73,High risk fund
2,3,24/02/2020,74,1.43,74,Moderate risk fund
3,4,24/02/2020,75,1.23,75,Government bond fund
4,5,24/02/2020,76,1.24,76,Investment fund


#### Dropping fund_id after merge

In [16]:
fundFundPrice=fundFundPrice.drop(['id_y'], axis=1)

In [8]:
investment = path+"/Data manipulation test - Investments.csv"
investment_path = pd.read_csv(investment)
investment_path.head()

,id,state,fund_id,fund_price_on,type,fund_units,amount_pennies,employee_id
0,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101
1,29,completed,73,2020-03-05,BuyInvestment,3400.000000,340000,98
2,30,completed,78,2020-02-24,BuyInvestment,121.830000,48732,101
3,31,rejected,73,NaN,BuyInvestment,0.000000,230000,99
4,32,completed,73,2020-02-24,BuyInvestment,114.294483,230000,99


#### Merging fund data with investment data on basis of fund_id

In [25]:
merge_Inv_Fund= pd.merge(investment_path, fundFundPrice, left_on='fund_id', right_on='fund_id', how='left')
merge_Inv_Fund.head(10)

,id,state,fund_id,fund_price_on,type,fund_units,amount_pennies,employee_id,id_x,amount_on,unit_value_pounds,Fund Name
0,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,2,24/02/2020,1.23,High risk fund
1,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,8,25/02/2020,1.29,High risk fund
2,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,14,26/02/2020,1.09,High risk fund
3,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,20,27/02/2020,1.40,High risk fund
4,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,26,28/02/2020,1.33,High risk fund


In [29]:
merge_Inv_Fund.describe().T

,count,mean,std,min,25%,50%,75%,max
id,34680.0,197.600000,88.940415,28.0,136.00,208.000000,272.000000,3.350000e+02
fund_id,34680.0,73.215686,0.976477,72.0,73.00,73.000000,73.000000,7.800000e+01
fund_units,34680.0,936.725765,4022.045686,0.0,1.64,28.573621,333.333333,3.585917e+04
amount_pennies,34680.0,98993.866667,464686.356811,0.0,7.00,12345.000000,40000.000000,4.203100e+06
employee_id,34680.0,2454.682353,5685.125525,36.0,97.00,226.000000,737.000000,1.861800e+04
id_x,34680.0,407.200000,235.557676,1.0,204.75,408.500000,612.250000,8.160000e+02
unit_value_pounds,34680.0,1.208420,0.177334,0.9,1.06,1.230000,1.360000,1.500000e+00


#### Merging data with employee data on basis of employee_id

In [30]:
merge_Inv_Fund_Emp= pd.merge(merge_Inv_Fund, employee_path, left_on='employee_id', right_on='id', how='left')


C:\Users\Anshika\AppData\Local\Temp/ipykernel_8556/1639652918.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge_Inv_Fund_Emp= pd.merge(merge_Inv_Fund, employee_path, left_on='employee_id', right_on='id', how='left')


In [32]:
merge_Inv_Fund_Emp.drop(['id_y'], axis=1)

,id_x,state,fund_id,fund_price_on,type,fund_units,amount_pennies,employee_id,id_x,amount_on,unit_value_pounds,Fund Name,gender,region,salary,DOB,industries
0,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101,2,24/02/2020,1.23,High risk fund,Female,Isle of Man,13000,23937,Transportation and storage
1,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101,8,25/02/2020,1.29,High risk fund,Female,Isle of Man,13000,23937,Transportation and storage
2,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101,14,26/02/2020,1.09,High risk fund,Female,Isle of Man,13000,23937,Transportation and storage
3,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101,20,27/02/2020,1.40,High risk fund,Female,Isle of Man,13000,23937,Transportation and storage
4,28,completed,73,2020-02-24,SellInvestment,243.660000,48732,101,26,28/02/2020,1.33,High risk fund,Female,Isle of Man,13000,23937,Transportation and storage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34675,335,completed,73,2020-08-10,BuyInvestment,0.091538,1000,18600,788,25/08/2020,0.91,High risk fund,Male,South West,79000,34561,Real estate activities
34676,335,completed,73,2020-08-10,BuyInvestment,0.091538,1000,18600,794,26/08/2020,1.20,High risk fund,Male,South West,79000,34561,Real estate activities
34677,335,completed,73,2020-08-10,BuyInvestment,0.091538,1000,18600,800,27/08/2020,0.90,High risk fund,Male,South West,79000,34561,Real estate activities
34678,335,completed,73,2020-08-10,BuyInvestment,0.091538,1000,18600,806,28/08/2020,1.28,High risk fund,Male,South West,79000,34561,Real estate activities


#### Finding the missing percentage in all the columns

In [33]:
missing_values_count=merge_Inv_Fund_Emp.isnull().sum()
print(f"Count of Missing Values\n\n {missing_values_count}")

Count of Missing Values

 id_x                    0
state                   0
fund_id                 0
fund_price_on        8160
type                    0
fund_units              0
amount_pennies          0
employee_id             0
id_x                    0
amount_on               0
unit_value_pounds       0
Fund Name               0
id_y                    0
gender                408
region                  0
salary                  0
DOB                     0
industries              0
dtype: int64


#### Creating a single data file as Merged data

In [36]:
merge_Inv_Fund_Emp.to_csv('Merged_Data.csv')

In [47]:
merge_Inv_Fund_Emp.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34680 entries, 0 to 34679
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_x               34680 non-null  int64         
 1   state              34680 non-null  object        
 2   fund_id            34680 non-null  int64         
 3   fund_price_on      26520 non-null  datetime64[ns]
 4   type               34680 non-null  object        
 5   fund_units         34680 non-null  float64       
 6   amount_pennies     34680 non-null  int64         
 7   employee_id        34680 non-null  int64         
 8   id_x               34680 non-null  int64         
 9   amount_on          34680 non-null  datetime64[ns]
 10  unit_value_pounds  34680 non-null  float64       
 11  Fund Name          34680 non-null  object        
 12  id_y               34680 non-null  int64         
 13  gender             34272 non-null  object        
 14  region

#### Changing the data type of amount_on and fund_price_on from object to date

In [46]:
merge_Inv_Fund_Emp['amount_on'] = pd.to_datetime(merge_Inv_Fund_Emp['amount_on'], format='%d/%m/%Y')
merge_Inv_Fund_Emp['fund_price_on'] = pd.to_datetime(merge_Inv_Fund_Emp['fund_price_on'], format='%Y-%m-%d')

merge_Inv_Fund_Emp.head()

,id_x,state,fund_id,fund_price_on,type,fund_units,amount_pennies,employee_id,id_x,amount_on,unit_value_pounds,Fund Name,id_y,gender,region,salary,DOB,industries
0,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,2,2020-02-24,1.23,High risk fund,101,Female,Isle of Man,13000,23937,Transportation and storage
1,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,8,2020-02-25,1.29,High risk fund,101,Female,Isle of Man,13000,23937,Transportation and storage
2,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,14,2020-02-26,1.09,High risk fund,101,Female,Isle of Man,13000,23937,Transportation and storage
3,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,20,2020-02-27,1.40,High risk fund,101,Female,Isle of Man,13000,23937,Transportation and storage
4,28,completed,73,2020-02-24,SellInvestment,243.66,48732,101,26,2020-02-28,1.33,High risk fund,101,Female,Isle of Man,13000,23937,Transportation and storage


In [49]:
merge_Inv_Fund_Emp.to_csv('Merged_Data.csv')

### filter on the basis of date range given in the assessment

In [54]:
merge_Inv_Fund_Emp_temp = merge_Inv_Fund_Emp.loc[(merge_Inv_Fund_Emp['amount_on']>= '2020-02-24') & (merge_Inv_Fund_Emp['amount_on']<= '2020-08-31') & (merge_Inv_Fund_Emp['state']== 'completed')]

### Question 1

In [65]:
x= merge_Inv_Fund_Emp_temp.groupby(['employee_id','amount_on']).agg({'fund_units':'sum', 'unit_value_pounds':'sum'})


In [66]:
x.head()

fund_units  unit_value_pounds
employee_id amount_on                                
36          2020-02-24    5.179769               4.92
            2020-02-25    5.179769               5.16
            2020-02-26    5.179769               4.36
            2020-02-27    5.179769               5.60
            2020-02-28    5.179769               5.32

## question 2

In [72]:
Unit_price= merge_Inv_Fund_Emp_temp.groupby(['amount_on','fund_id']).agg({'unit_value_pounds':'sum'})


In [73]:
Unit_price.head()

unit_value_pounds
amount_on  fund_id                   
2020-02-24 72                   25.92
           73                  189.42
           74                    7.15
           75                    8.61
           76                    8.68

## question 3

In [74]:
merge_Inv_Fund_Emp_temp["AUM"] = merge_Inv_Fund_Emp_temp["fund_units"] * merge_Inv_Fund_Emp_temp["unit_value_pounds"]


C:\Users\Anshika\AppData\Local\Temp/ipykernel_8556/3611375485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_Inv_Fund_Emp_temp["AUM"] = merge_Inv_Fund_Emp_temp["fund_units"] * merge_Inv_Fund_Emp_temp["unit_value_pounds"]


In [82]:
merge_Inv_Fund_Emp_temp.groupby(['amount_on','fund_id'])[['AUM']].sum()


AUM
amount_on  fund_id               
2020-02-24 72        87682.016012
           73       104859.380047
           74         1712.037087
           75        89941.336642
           76         4642.217419
...                           ...
2020-08-31 73        86956.559064
           74         1089.478146
           75        89210.106263
           76         3668.849250
           78         3764.825100

[816 rows x 1 columns]

## question 4

In [83]:
merge_Inv_Fund_Emp_temp.groupby(['amount_on','fund_id','Fund Name'])[['AUM']].sum()


AUM
amount_on  fund_id Fund Name                          
2020-02-24 72      Low risk fund          87682.016012
           73      High risk fund        104859.380047
           74      Moderate risk fund      1712.037087
           75      Government bond fund   89941.336642
           76      Investment fund         4642.217419
...                                                ...
2020-08-31 73      High risk fund         86956.559064
           74      Moderate risk fund      1089.478146
           75      Government bond fund   89210.106263
           76      Investment fund         3668.849250
           78      Mixed fund              3764.825100

[816 rows x 1 columns]

## question 5

In [89]:
merge_Inv_Fund_Emp_temp.groupby(['gender'])[['salary']].median()


,salary
gender,
Female,62000.0
Male,75000.0


In [106]:
merge_Inv_Fund_Emp_temp.groupby(['gender'])[['salary']].mean()


,salary
gender,
Female,66764.705882
Male,89054.794521


#### Men earn more than female as female mean salary is lesser than male mean salary

## question 6

In [100]:
count =merge_Inv_Fund_Emp_temp.groupby(['gender'])['salary'].count()

gender
Female    16184
Male       9928
Name: salary, dtype: int64

#### finding the difference in the salary of male and female

In [116]:
merge_Inv_Fund_Emp_temp.loc[(merge_Inv_Fund_Emp_temp['gender']== 'Male')].groupby(['gender'])['salary'].mean()[0]-merge_Inv_Fund_Emp_temp.loc[(merge_Inv_Fund_Emp_temp['gender']== 'Female')].groupby(['gender'])['salary'].mean()[0]

22290.08863819501

## question 7

In [136]:
merge_Inv_Fund_Emp_temp.loc[(merge_Inv_Fund_Emp_temp['gender']== 'Female')].groupby(['industries','gender'])['gender'].count().reset_index(name='count').sort_values(['count'],ascending=False).head()

,industries,gender,count
8,"Electricity, gas, steam and air conditioning s...",Female,3536
14,Real estate activities,Female,2176
15,Transportation and storage,Female,2176
16,"Water supply, sewerage, waste management and r...",Female,1496
5,"Arts, entertainment and recreation",Female,1224


#### These are the top 5 industries where most number of females are employed

### question 8


#### Having a model will be a very be extremely helpful in order to predict the pay gap. Model can be build after removing the null date. Columns or features which have outliers should be removed using DBSCAN or Isolation Forest as outliers can create noise in the data which would not be useful in our analysis. Highly Co-related columns should dropped as they all tend to given same information. For model creation we need to make sure data is converted into integer as categorical data cannot be processed and do not help in finding trends, thereofre, label or ordinal encoders can be used. Class balance methods like SMOTE should be used to balance between minority/majority classes. Feature selection is another way to minimize the number of extremely important features which are very useful in our analysis like employee_id, industries, salary, fund_id etc. 

#### Then we can split the data into train and test using stratified k-fold and apply models like decision tree, random forest, svm, nlp, ensemble methods etc. For finding the best model we can evaluate using metrics like Accuracy, F1-Score, Recall, Precision, AUC. 

#### For model prediction we can also divide the dataset on the basis of salary ranges like people with low salary bucket, medium bucket and high bucket. 